# Import Libraries

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#Load and Label Data

In [4]:
df_fake = pd.read_csv("/content/Fake.csv")
df_true = pd.read_csv("/content/True.csv")

df_fake["class"] = 0
df_true["class"] = 1


#Separate Manual Testing Data#

In [5]:
df_fake_manual_testing = df_fake.tail(10)
df_true_manual_testing = df_true.tail(10)

df_fake = df_fake.iloc[:-10]
df_true = df_true.iloc[:-10]


#Merge and Shuffle Data

In [6]:
df_merge = pd.concat([df_fake, df_true], axis=0)
df_merge = df_merge.sample(frac=1).reset_index(drop=True)


#Text Preprocessing Function

In [7]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)


#Preprocessing to Text Data

In [8]:
df_merge["text"] = df_merge["text"].apply(preprocess_text)


#Split Data into Training and Testing Sets

In [9]:
X = df_merge["text"]
y = df_merge["class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)


#Convert Text Data to TF-IDF Features

In [10]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=5)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

#Train Logistic Regression with Grid Search

In [11]:
param_grid = {"C": [0.01, 0.1, 1, 10, 100]}
lr = LogisticRegression(max_iter=2000)

grid_search = GridSearchCV(lr, param_grid, cv=5, scoring="accuracy")
grid_search.fit(X_train_tfidf, y_train)

best_lr = grid_search.best_estimator_


#Evaluate the Model

In [12]:
pred_lr = best_lr.predict(X_test_tfidf)

print(f"Best Logistic Regression Model: {grid_search.best_params_}")
print(f"Accuracy: {accuracy_score(y_test, pred_lr):.4f}")
print(classification_report(y_test, pred_lr))


Best Logistic Regression Model: {'C': 100}
Accuracy: 0.9912
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5868
           1       0.99      0.99      0.99      5352

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



#Predicting test data

In [16]:
import joblib

joblib.dump(best_lr, "logistic_regression_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

best_lr = joblib.load("logistic_regression_model.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

def predict_news(article):
    article_tfidf = vectorizer.transform([article])
    prediction = best_lr.predict(article_tfidf)
    return "Fake News" if prediction[0] == 0 else "Real News"

sample_fake_articles = df_fake_manual_testing["text"].tolist()
sample_true_articles = df_true_manual_testing["text"].tolist()

test_articles = sample_fake_articles + sample_true_articles
ground_truth_labels = ["Fake News"] * len(sample_fake_articles) + ["Real News"] * len(sample_true_articles)

for i, (article, actual_label) in enumerate(zip(test_articles, ground_truth_labels), 1):
    predicted_label = predict_news(article)
    print(f"Article {i}: Predicted: {predicted_label} | Actual: {actual_label}")


Article 1: Predicted: Fake News | Actual: Fake News
Article 2: Predicted: Fake News | Actual: Fake News
Article 3: Predicted: Fake News | Actual: Fake News
Article 4: Predicted: Fake News | Actual: Fake News
Article 5: Predicted: Fake News | Actual: Fake News
Article 6: Predicted: Fake News | Actual: Fake News
Article 7: Predicted: Fake News | Actual: Fake News
Article 8: Predicted: Fake News | Actual: Fake News
Article 9: Predicted: Fake News | Actual: Fake News
Article 10: Predicted: Fake News | Actual: Fake News
Article 11: Predicted: Real News | Actual: Real News
Article 12: Predicted: Real News | Actual: Real News
Article 13: Predicted: Real News | Actual: Real News
Article 14: Predicted: Real News | Actual: Real News
Article 15: Predicted: Real News | Actual: Real News
Article 16: Predicted: Real News | Actual: Real News
Article 17: Predicted: Real News | Actual: Real News
Article 18: Predicted: Real News | Actual: Real News
Article 19: Predicted: Real News | Actual: Real News
Ar

#Predicting general articles

In [14]:
art=["Trump Declares War on Mainstream Media","Breaking reports suggest that President Joe Biden has decided to step down from office due to severe health complications. Sources claim that Vice President Kamala Harris will take over immediately, although no official announcement has been made yet.","WASHINGTON (Reuters) - Lawmakers are in tense negotiations as Congress debates whether to raise the U.S. debt ceiling. With the deadline approaching, Democrats and Republicans remain divided on key fiscal policies."]
for i, article in enumerate(art, 1):
    print(f"Article {i}: {predict_news(article)}")

Article 1: Fake News
Article 2: Fake News
Article 3: Real News
